In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from collections import Counter
from keras.preprocessing.text import Tokenizer
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding,ConvLSTM2D
from keras import layers

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# !pip install s2sphere
# import s2sphere

# r = s2sphere.RegionCoverer()
# p1 = s2sphere.LatLng.to_point()

In [ ]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

     | 399kB 3.7MB/s
  Created wheel for tqdm: filename=tqdm-4.28.1-py2.py3-none-any.whl size=47867 sha256=66ae0d259a13d58e342156da9dd11f01736760eb9032be6f2d2fa53912a54174
  Stored in directory: /tmp/pip-ephem-wheel-cache-ch6d_4i7/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
os.chdir("/content/drive/My Drive/")

In [ ]:
bus= pd.read_excel("Retail_final_sale.xlsx")

In [ ]:
bus.head()

,Location,Area,Area code,Price(in Euros),House_type,Beds,Bathroom,Area_code,floor_area,Entered,Views,wakable_distance,property_latitude,property_longitude,aminities_latitude,aminities_longitude,score,Company Name,Address,Zip/Postal,Code,Type,Rating
0,1 Kirkpatrick House|| Spencer House|| IFSC|| D...,Dublin 1,D01,285000,Apartment for sale,1,1,D00,42.1,31.05.2020,2108.0,0.8,53.350082,-6.238685,53.348536,-6.229301,0.75,Centra Point Campus,"Centra Point Campus, Mayor Street Upper, Point...",D01,NaN,Retail store,3.1
1,127 Scarriff House|| Custom House Square|| Dub...,Dublin 1,D01,350000,Apartment for sale,2,1,D01,59.0,28.05.2020,1883.0,0.9,53.349712,-6.241361,53.348536,-6.229301,0.75,Centra Point Campus,"Centra Point Campus, Mayor Street Upper, Point...",D01,NaN,Retail store,3.1
2,14 Spencer House|| Custom House Square|| IFSC|...,Dublin 1,D01,370000,Apartment for sale,2,2,D01,75.0,30.05.2020,4866.0,0.9,53.349209,-6.242093,53.348536,-6.229301,0.75,Centra Point Campus,"Centra Point Campus, Mayor Street Upper, Point...",D01,NaN,Retail store,3.1
3,147 Castleforbes Square|| Docklands|| Dublin 1,Dublin 1,D01,375000,Apartment for sale,2,1,D00,70.1,27.11.2019,5008.0,0.2,53.348516,-6.231471,53.348536,-6.229301,0.75,Centra Point Campus,"Centra Point Campus, Mayor Street Upper, Point...",D01,NaN,Retail store,3.1
4,30 Kirkpatrick House|| Spencer Dock|| Dublin 1,Dublin 1,D01,300000,Apartment for sale,1,1,D01,40.4,31.05.2020,1268.0,0.7,53.349818,-6.237930,53.348536,-6.229301,0.75,Centra Point Campus,"Centra Point Campus, Mayor Street Upper, Point...",D01,NaN,Retail store,3.1


In [ ]:
cols = [col for col in bus.columns if col in ['wakable_distance']]
ground_truth = bus[cols]
# 'aminities_latitude','aminities_longitude'

In [ ]:
ground_truth

,wakable_distance
0,0.8
1,0.9
2,0.9
3,0.2
4,0.7
...,...
9313,0.7
9314,0.6
9315,0.9
9316,0.6


In [ ]:
cols = [col for col in bus.columns if col in ['property_latitude','property_longitude']]
property_location = bus[cols]
# 'property_latitude','property_longitude'

In [ ]:
property_location

,property_latitude,property_longitude
0,53.350082,-6.238685
1,53.349712,-6.241361
2,53.349209,-6.242093
3,53.348516,-6.231471
4,53.349818,-6.237930
...,...,...
9313,53.272551,-6.368175
9314,53.280152,-6.385961
9315,53.284715,-6.376575
9316,53.276323,-6.411363


In [ ]:
property_arr=pd.concat([property_location['property_latitude'],property_location['property_longitude']],axis=1)
# ,property_location['property_longitude'
gt_arr=pd.concat([ground_truth['wakable_distance']],axis=1)
# ,ground_truth['aminities_longitude'

In [ ]:
test_set_retail_tmp= pd.read_excel("Retail_cluster_test_set.xlsx")
test_set_retail_tmp

,Cluster Labels,Location,Retail_stores,Area,Area code,property_latitude,property_longitude
0,1,1 Howth Lodge|| Howth|| Dublin 13,0,Dublin 13,D13,53.391055,-6.084403
1,1,1 Iona Court|| Iona Villas|| Glasnevin|| Dublin 9,0,Dublin 9,D09,53.368056,-6.265257
2,1,1 Kiltipper Rise|| Kiltipper|| D24VAF4|| Talla...,0,Dublin 24,D24,53.269783,-6.371204
3,1,10 Airpark Close|| Rathfarnham|| Dublin 16,0,Dublin 16,D16,53.274776,-6.303783
4,1,10 Cedar Grove|| Templeogue|| Dublin 6w,0,Dublin 6,D06,53.289345,-6.321807
...,...,...,...,...,...,...,...
358,1,Wilford House|| Shankill|| Dublin 18,0,Dublin 18,D18,53.214227,-6.121371
359,1,Windward|| 93 Barton Road East|| Churchtown|| ...,0,Dublin 14,D14,53.287351,-6.256489
360,1,Woodberry|| The Birches|| Torquay Road|| Foxro...,0,Dublin 18,D18,53.270212,-6.183716
361,1,Woodfield|| Westminster Road|| Foxrock|| Dubli...,0,Dublin 18,D18,53.266892,-6.182433


In [ ]:
cols = [col for col in test_set_retail_tmp.columns if col in ['property_latitude','property_longitude']]
test_set_retail = test_set_retail_tmp[cols]
test_set_retail
# 'property_latitude','property_longitude'

,property_latitude,property_longitude
0,53.391055,-6.084403
1,53.368056,-6.265257
2,53.269783,-6.371204
3,53.274776,-6.303783
4,53.289345,-6.321807
...,...,...
358,53.214227,-6.121371
359,53.287351,-6.256489
360,53.270212,-6.183716
361,53.266892,-6.182433


In [ ]:
cap_train, cap_test, gt_train, gt_test = train_test_split(property_arr, gt_arr, test_size=0.10, random_state=45)

In [ ]:
cap_train

,property_latitude,property_longitude
9253,53.290313,-6.352460
1347,53.345597,-6.264016
7563,53.382755,-6.239557
6325,53.398884,-6.213807
4460,53.331633,-6.260634
...,...,...
8772,53.386856,-6.376639
163,53.347292,-6.267458
6012,53.332566,-6.289277
6558,53.317401,-6.299840


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=15,metric='haversine')
neigh.fit(cap_train, gt_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='haversine',
                    metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                    weights='uniform')

In [ ]:
predVal=neigh.predict(cap_test)

In [ ]:
predVal=neigh.predict(test_set_retail)

In [ ]:
# gt_test

In [ ]:
predVal

array([[0.5584    ],
       [0.66666667],
       [0.64      ],
       [0.72666667],
       [0.5306    ],
       [0.72      ],
       [0.66      ],
       [0.70666667],
       [0.75333333],
       [0.62473333],
       [0.68      ],
       [0.65333333],
       [0.76      ],
       [0.39686667],
       [0.79333333],
       [0.54446667],
       [0.74      ],
       [0.58666667],
       [0.68      ],
       [0.67333333],
       [0.75333333],
       [0.60666667],
       [0.74666667],
       [0.66      ],
       [0.78      ],
       [0.66666667],
       [0.435     ],
       [0.79333333],
       [0.444     ],
       [0.72      ],
       [0.60666667],
       [0.66      ],
       [0.55333333],
       [0.63333333],
       [0.69333333],
       [0.62973333],
       [0.59606667],
       [0.59333333],
       [0.70666667],
       [0.56666667],
       [0.58666667],
       [0.5654    ],
       [0.8       ],
       [0.66      ],
       [0.6       ],
       [0.61333333],
       [0.3912    ],
       [0.613

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(gt_test, predVal))
print('Mean Squared Error:', metrics.mean_squared_error(gt_test, predVal))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(gt_test, predVal)))
# print(neigh.score(predVal,gt_test)),

Mean Absolute Error: 0.17547510729613733
Mean Squared Error: 0.045626125188364335
Root Mean Squared Error: 0.21360272748343906


In [ ]:
# mat=[predVal,gt_test]
# df=pd.DataFrame(mat).transpose()
# df.columns=('h0','y')
# df1=pd.DataFrame(df)
# df1.rename(columns={'h0':'h1','y':'y1'},inplace=True)
# df1['diff']=np.where(df.h0==df1.y1,1,0)
# df1

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 150)
RFregressor.fit(cap_train, gt_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=150, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
predictions_caption_DT = RFregressor.predict(cap_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(gt_test, predictions_caption_DT))
print('Mean Squared Error:', metrics.mean_squared_error(gt_test, predictions_caption_DT))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(gt_test, predictions_caption_DT)))

Mean Absolute Error: 0.1842309196405646
Mean Squared Error: 0.053010364068602406
Root Mean Squared Error: 0.23023979688273355


In [ ]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(cap_train, gt_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
predictions_caption_DT = RFregressor.predict(cap_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(gt_test, predictions_caption_DT))
print('Mean Squared Error:', metrics.mean_squared_error(gt_test, predictions_caption_DT))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(gt_test, predictions_caption_DT)))

Mean Absolute Error: 0.1842309196405646
Mean Squared Error: 0.053010364068602406
Root Mean Squared Error: 0.23023979688273355


In [ ]:
# frame =[property_location,test_set_retail]
# propertyarr = pd.concat(frame)
# propertyarr

In [ ]:
predictions_caption_DT = RFregressor.predict(test_set_retail)

In [ ]:
# import statsmodels.api as sm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# sm.graphics.tsa.plot_acf(bus.property_longitude, lags=40)
# plt.show()

In [ ]:
lon1=53.3910545
lat1=-6.084403244
radius=0.58666667
lon1, lat1, lon2 = map(math.radians, [lon1, lat1, lon1])
#latitude is changing, not longitude, so dlon = 0
dlon = 0
c = radius / 6367 / 0.621363
a = (math.sin(c/2))**2
print(a)

5.497463585602047e-09


In [ ]:
find_latitude_range(53.3910545,-6.084403244,0.58666667)

In [ ]:
pip install pyclj

  Created wheel for pyclj: filename=pyclj-0.4.0-cp36-none-any.whl size=5203 sha256=15d3e1c8c535209fdfc25964ac98efbef81a1ed82aabb1c19faede517849158f
  Stored in directory: /root/.cache/pip/wheels/54/87/41/ad25c3054c96079815116b593edcdd757d69d8fda6af5b670a
Successfully built pyclj


In [ ]:
angdist = (0.58666667/6378.137)
lat1 = math.radians(53.3910545)
lon1=math.radians(-6.084403244)

In [ ]:
lat2=math.degrees((math.asin(((math.sin(lat1))*(math.cos(angdist)))+((math.cos(lat1))*(math.sin(angdist))))))
lon2=(math.degress(lon1+(math.atan2((math.sin(angdist)*(math.cos(lat1))*((math.cos(angdist)-((math.sin(lat1))*(math.sin(lat2))))))))
lon2

SyntaxError: ignored